In [1]:
import os
os.chdir('../')

In [55]:
import pandas as pd
import warnings
from source.lib.helpers import *
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [56]:
def WeightedQuantile(values, weights, quantiles):
    values = np.array(values)
    weights = np.array(weights)
    sorter = np.argsort(values)
    values = values[sorter]
    weights = weights[sorter]
    cumulative_weight = np.cumsum(weights)
    total_weight = cumulative_weight[-1]
    return np.interp(np.array(quantiles) * total_weight, cumulative_weight, values)

def Assign3Bin(repo):
    val = base_wm[repo]
    if val <= q33:
        return 0
    elif val <= q67:
        return 1
    else:
        return 2


In [57]:
# controls that I can add down the road
# % of problems that are unlinked prs/linked


df_problems_contr_filtered = pd.read_parquet('issue/filtered_problem_data.parquet')
df_problems_contr_filtered['departed_involved'] = df_problems_contr_filtered.apply(lambda x: x['departed_actor_id'] in x['all_actors'], axis = 1)
df_problems_contr_filtered['key_contributor_count'] = df_problems_contr_filtered['important_actors_rolling'].apply(len)
df_problems_contr_filtered['total_contributor_count'] = df_problems_contr_filtered['all_actors_period'].apply(len)
df_problems_contr_filtered['departed_opener'] = pd.to_numeric(df_problems_contr_filtered.apply(lambda x: x['departed_actor_id'] in x['pr_opener'] if x['departed_involved'] else np.nan, axis = 1))
df_problems_contr_filtered['departed_author'] = pd.to_numeric(df_problems_contr_filtered.apply(lambda x: x['departed_actor_id'] in x['pr_authors'] if x['departed_involved'] else np.nan, axis = 1))

df_project_filtered_group = df_problems_contr_filtered.groupby(
    ['repo_name', 'time_period', 'treatment_period', 'key_contributor_count', 'total_contributor_count']
).agg(
    problem_count=('problem_id', 'count'),
    ind_collab=('ind_collab_roll', 'mean'),
    ind_key_collab=('ind_key_collab_roll', 'mean'),
    ind_other_collab=('ind_other_collab_roll', 'mean'),
    departed_involved_count=('departed_involved','sum'),
    departed_involved=('departed_involved','mean'),
    departed_opened_count=('departed_opener','sum'),
    departed_opened=('departed_opener','mean'), # conditional on involvement
    departed_authored_count=('departed_author','sum'),
    departed_authored=('departed_author','mean'), # conditional on involvement
).reset_index()

In [64]:
df_project_filtered_group

,repo_name,time_period,treatment_period,key_contributor_count,total_contributor_count,problem_count,ind_collab,ind_key_collab,ind_other_collab,departed_involved_count,departed_involved,departed_opened_count,departed_opened,departed_authored_count,departed_authored,ind_collab_2bin,ind_collab_3bin,ind_key_collab_2bin,ind_key_collab_3bin,ind_other_collab_2bin,ind_other_collab_3bin,departed_involved_2bin,departed_involved_3bin,key_contributor_count_2bin,key_contributor_count_3bin,total_contributor_count_2bin,total_contributor_count_3bin,problem_count_2bin,problem_count_3bin,departed_opened_2bin,departed_opened_3bin,departed_authored_2bin,departed_authored_3bin,departed_involved_count_2bin,departed_involved_count_3bin
0,AnalogJ/lexicon,2016-01-01,2020-01-01,2,14,44,0.636364,0.933333,0.000000,30,0.681818,0.0,0.000000,0.0,0.000000,1,2,1,2,0,0,1,2,0,0,0,1,0,1,0,0,0,0,1,2
1,AnalogJ/lexicon,2016-07-01,2020-01-01,2,29,42,0.547619,0.958333,0.000000,24,0.571429,0.0,0.000000,0.0,0.000000,1,2,1,2,0,0,1,2,0,0,0,1,0,1,0,0,0,0,1,2
2,AnalogJ/lexicon,2017-01-01,2020-01-01,2,29,60,0.366667,0.880000,0.000000,25,0.416667,0.0,0.000000,0.0,0.000000,1,2,1,2,0,0,1,2,0,0,0,1,0,1,0,0,0,0,1,2
3,AnalogJ/lexicon,2017-07-01,2020-01-01,2,27,29,0.448276,0.812500,0.000000,16,0.551724,0.0,0.000000,0.0,0.000000,1,2,1,2,0,0,1,2,0,0,0,1,0,1,0,0,0,0,1,2
4,AnalogJ/lexicon,2018-01-01,2020-01-01,4,65,99,0.585859,0.867925,0.260870,53,0.535354,0.0,0.000000,0.0,0.000000,1,2,1,2,0,0,1,2,0,0,0,1,0,1,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,xapi-project/xen-api,2021-07-01,2022-01-01,6,24,136,0.816176,1.000000,0.725275,45,0.330882,16.0,0.355556,16.0,0.355556,1,2,1,2,1,0,1,2,1,2,0,0,0,1,1,1,1,1,1,2
1321,xapi-project/xen-api,2022-01-01,2022-01-01,5,20,154,0.883117,NaN,0.883117,0,0.000000,0.0,NaN,0.0,NaN,1,2,1,2,1,0,1,2,1,2,0,0,0,1,1,1,1,1,1,2
1322,xapi-project/xen-api,2022-07-01,2022-01-01,4,20,145,0.834483,NaN,0.834483,0,0.000000,0.0,NaN,0.0,NaN,1,2,1,2,1,0,1,2,1,2,0,0,0,1,1,1,1,1,1,2
1323,xapi-project/xen-api,2023-01-01,2022-01-01,4,29,218,0.711009,NaN,0.711009,0,0.000000,0.0,NaN,0.0,NaN,1,2,1,2,1,0,1,2,1,2,0,0,0,1,1,1,1,1,1,2


In [62]:

preperiod_recent = df_project_filtered_group.query('time_period < treatment_period').groupby('repo_name').tail(5)
preperiod_recent['other_involved_count'] = preperiod_recent['departed_involved_count'] - preperiod_recent['problem_count']
preperiod_recent['uniform_weight'] = 1

count_dict = {
    'ind_collab': 'problem_count',
    'ind_key_collab': 'departed_involved_count',
    'ind_other_collab': 'other_involved_count',
    'departed_involved': 'problem_count',
    'departed_involved_count': 'uniform_weight',
    'key_contributor_count': 'uniform_weight',
    'total_contributor_count': 'uniform_weight',
    'problem_count': 'uniform_weight',
    'departed_opened': 'departed_opened_count',
    'departed_authored': 'departed_authored_count'
}

for collab_type, count_col in count_dict.items():
    avg_collab = WeightedMean(preperiod_recent[collab_type], preperiod_recent[count_col])
    base_wm = preperiod_recent.groupby('repo_name').apply(
        lambda df: WeightedMean(df[collab_type], df[count_col], zero_weight_return = 0)
    )

    above_set = set(base_wm[base_wm > avg_collab].index)
    df_project_filtered_group[f"{collab_type}_2bin"] = df_project_filtered_group['repo_name'].apply(lambda x: int(x in above_set))

    # 3-bin: weighted quantiles
    q33, q67 = WeightedQuantile(preperiod_recent[collab_type], preperiod_recent[count_col], [0.33, 0.67])

    df_project_filtered_group[f"{collab_type}_3bin"] = df_project_filtered_group['repo_name'].apply(Assign3Bin)

df_project_filtered_group.to_parquet('issue/project_collaboration.parquet')